# Análise exploratória de dados

## Carregar bibliotecas

In [2]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd

## Dicionário de dados

In [3]:
pd.read_csv('../data/external/dicionario.csv')

,variavel,significado,tipo,Unnamed: 3
0,id,ID único por usuário,nominal,NaN
1,tweet_text,Texto publicado,nominal,NaN
2,tweet_date,Data de publicação,nominal,NaN
3,sentiment,Algorítmo de classificação do sentimento do us...,nominal,"[0,1,2]"
4,query_used,Palavra relevante,nominal,"[':)', ':(', 'veja', 'jornaloglobo', 'g1', 'fo..."


## Leitura do arquivo de dados

In [4]:
db = pd.read_csv('../data/raw/data.csv', delimiter=";")
db.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1050785521201541121,@Laranjito76 A pessoa certa para isso seria o ...,Fri Oct 12 16:29:25 +0000 2018,1,:)
1,1050785431955140608,"@behin_d_curtain Para mim, é precisamente o co...",Fri Oct 12 16:29:04 +0000 2018,1,:)
2,1050785401248645120,Vou fazer um video hoje... estou pensando em f...,Fri Oct 12 16:28:56 +0000 2018,1,:)
3,1050785370982547461,"aaaaaaaa amei tanto essas polaroids, nem sei e...",Fri Oct 12 16:28:49 +0000 2018,1,:)
4,1050785368902131713,Valoriza o coração do menininho que vc tem. El...,Fri Oct 12 16:28:49 +0000 2018,1,:)


## 1. Descrição dos dados

In [6]:
# Estatística de cada coluna 
db.describe()

,id,sentiment
count,1.000000e+05,100000.000000
mean,1.045736e+18,1.000000
std,4.262747e+15,0.816497
min,1.027374e+18,0.000000
25%,1.045369e+18,0.000000
50%,1.046820e+18,1.000000
75%,1.049106e+18,2.000000
max,1.053355e+18,2.000000


In [7]:
# Quantidade de linhas x colunas
db.shape

(100000, 5)

A base de dados possui $1 \times 10^5$ instâncias e cinco colunas, das quais três correspondem a variáveis discretas e duas a variáveis qualitativas.

In [8]:
# Nome das colunas que representam as variáveis
db.columns

Index(['id', 'tweet_text', 'tweet_date', 'sentiment', 'query_used'], dtype='object')

In [9]:
#Tipo e contagem por coluna
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          100000 non-null  int64 
 1   tweet_text  100000 non-null  object
 2   tweet_date  100000 non-null  object
 3   sentiment   100000 non-null  int64 
 4   query_used  100000 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.8+ MB


In [10]:
# Número de valores nulos
db.isnull().sum()

id            0
tweet_text    0
tweet_date    0
sentiment     0
query_used    0
dtype: int64

A base de dados não possui valores nulos. 

### 1.1. Obtenção dos períodos do dia

In [ ]:
db['date'] = pd.to_datetime(db['tweet_date'])
db['period'] = (db['date'].dt.hour % 24 + 4) // 4
db['period'].replace({1: 'Late Night',
                      2: 'Early Morning',
                      3: 'Morning',
                      4: 'Noon',
                      5: 'Evening',
                      6: 'Night'}, inplace=True)
db['sentiment'].replace({0: 'Negative',
                         1: 'Positive',
                         2: 'Neutral'}, inplace=True)
db.head()

### 1.2. Quantidade de tweets por período

In [ ]:
sns.set_style('whitegrid')
axis = sns.histplot(data=db, y='period')
axis.set(frame_on=False)
axis.axvline(0, color='k', clip_on=False)
plt.ylabel('Period of the day')
plt.xlabel('Number of tweets')
plt.title('Tweets per period of the day')
plt.show()

Baseado no gráfico, a maior parte dos tweets concentram-se no período da tarde e madrugada. Veremos agora a relação entre os tweets positivos, negativos e neutros durante o período a partir do gráfico a seguir:

In [ ]:
sns.set_style('whitegrid')
axis = sns.histplot(data=db, y='period', hue='sentiment', bins=5,
                   palette={'Negative': 'red', 'Neutral': 'gray' , 'Positive': 'green'},
                   multiple='dodge')
axis.set(frame_on=False)
axis.axvline(0, color='k', clip_on=False)
plt.ylabel('Period of the day')
plt.xlabel('Number of tweets')
plt.title('Tweets per period of the day according to sentiment')
plt.show()

Observa-se que durante a madrugada a maioria dos tweets tendem a ser negativos. No meio-dia os positivos estão ligeiramente à frente dos negativos e durante o período noturno tweets negativos são inexistentes, sendo em sua maioria neutros.

## 2. Perguntas de partida e hipóteses

Com base na análise dos dados referentes a classificação de sentimentos do twitter, surgem as seguintes perguntas:
   1. Por que alguns períodos do dia possuem mais tweets do que outros?
   
   2. Qual a correlação entre o tipo de sentimento do usuário e o período do dia?
   
   3. O conteúdo do tweet é relevante para a avaliação do sentimento?
    
Como hipóteses temos:
  1. A disponibilidade das pessoas para realizar tweets varia conforme o tempo livre. No final da tarde, que para a grande maioria corresponde ao fim do expediente de trabalho/estudantil, os usuários possuem maior tempo para usufruirem das mídias socias. Assim como também no final da noite que pode ser um horário mais confortável para escrever sem se preocupar com afazeres posteriores.
  
  2. Ao acordar, muitas pessoas estão com pouca disposição e precisam lidar com a expectativa de iniciar algum tipo de trabalho ou atividade análoga, causando uma maioria de sentimentos negativos e neutros. Durante a tarde, metade do dia se foi e os sentimentos positivos aumentam e pela noite tendem a desaparecer devido ao encerramento das atividades. Já na madrugada, vemos uma maioria esmagadora de sentimentos negativos tendo em vista que uma parcela minoritária da população se encontra acordada nesse horário e muito provavelmente devem sofrer de insônia e doenças relacionadas que podem causar sentimentos negativos.
  
  3. O conteúdo pode não ter muita relação, mas a palavra-chave utilizada pelo banco de dados nos mostra uma tendência sentimental em relação ao conteúdo dos tweets que a utilizam.

## 3. Insights

Como observado nos gráficos, nos períodos de fim de tarde e noturno os sentimentos negativos dos usuários do twitter diminuem progressivamente até praticamente inexistirem após altas horas da noite.

Esses dados podem ser úteis para detectar que tipos de usuários podem estar sofrendo com insônia e estresse. É possível notar, a partir dessa análise, não somente informações do dia-a-dia dos usuários, mas também extrair informações valiosas a respeito do seu perfil e direcionar conteúdos de propaganda que sejam relevantes naquele momento do seu dia.